In [17]:
#import all necessary libraries to get more structured data 
import time
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
#import all necessary libraries to get more structured data 
import time
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from webdriver_manager.chrome import ChromeDriverManager
driver=webdriver.Chrome() 
website="https://www.dealstreetasia.com/section/venture-capital"  
driver.get(website)

# Define the exact date limit (2 months ago)
two_months_ago = datetime.now() - timedelta(days=60)

# Create CSV file for writing
csv_filename = "full_scraped_articles.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Source", "Date", "Article Content", "URL"])  # CSV Header

    # Function to parse and reformat date
    def parse_and_format_date(date_str):
        try:
            parsed_date = datetime.strptime(date_str, "%d %B, %Y")  # Convert '14 January, 2025'
            return parsed_date.strftime("%d ,%m, %Y")  # Format to '14 ,01, 2025'
        except ValueError:
            return None  # Handle unexpected formats

    last_article_old = False  # Stop when an article older than 2 months is found
    last_scraped_index = 0  # Keep track of last scraped article index

    while not last_article_old:
        # Step 1: Extract all current article links
        articles = driver.find_elements(By.XPATH, '//*[@id="archive-wrapper"]/div[4]/div[1]/div/div')
        article_links = [article.find_element(By.XPATH, './div[1]/a').get_attribute('href') for article in articles]

        # Start from the last scraped index (continue from where we left off)
        for i in range(last_scraped_index, len(article_links)):
            link = article_links[i]
            driver.get(link)
            time.sleep(2)  # Wait for article page to load

            try:
                # Extract article details
                title = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/h1').text.strip()
                source = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/span/a').text.strip()
                date_text = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/p').text.strip()
                body = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[2]/div[1]/article').text.strip().replace("\n", " ")

                # Convert date and compare with the 2-month limit
                formatted_date = parse_and_format_date(date_text)
                article_date = datetime.strptime(formatted_date, "%d ,%m, %Y") if formatted_date else None

                if article_date and article_date < two_months_ago:
                    print(f"⏹️ Stopping: Found an article older than 2 months ({formatted_date})")
                    last_article_old = True
                    break  # Stop processing further

                # Save to CSV
                writer.writerow([title, source, formatted_date, body, link])
                print(f"Saved article: {title} - {formatted_date}")

            except Exception as e:
                print(f" Error extracting article data from {link}: {e}")

            # Update last scraped index
            last_scraped_index = i + 1

            # Go back to the main page to scrape the next article
            driver.back()
            time.sleep(2)  # Wait for the main page to reload

        if last_article_old:
            break  # Stop clicking "More" if we found an old article

        # Step 2: Click the "More" button to load more articles
        try:
            more_button = driver.find_element(By.XPATH, '//*[@id="archive-wrapper"]/div[5]/div/button')
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(5)  # Wait for new articles to load

            # Update last_scraped_index to continue from where we left off
            last_scraped_index = len(article_links)

        except (NoSuchElementException, ElementClickInterceptedException):
            print(" No 'More' button found or can't be clicked. Stopping.")
            break  # Stop if the button is missing or can't be clicked

print(f"\n Scraping complete! Data saved to {csv_filename}")
driver.quit()


In [7]:
driver=webdriver.Chrome() 


In [8]:
website="https://www.dealstreetasia.com/section/venture-capital"  
driver.get(website)

found out all requirement paths to construct the scraping scripts

In [ ]:

# //*[@id="archive-wrapper"]/div[4]/div[1]/div/div[1]
# //*[@id="archive-wrapper"]/div[4]/div[1]/div/div[1]/div[1]/a ...link adress
# //*[@id="disable-copy"]/h1...title
# //*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/span/a ...source
# //*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/p....date
# //*[@id="disable-copy"]/div[2]/div[2]/div[1]/article...body 
# //*[@id="archive-wrapper"]/div[4]/div[1]/div/div[2]
# //*[@id="disable-copy"]/h1 ...title
# //*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/p ...date
# //*[@id="disable-copy"]/div[2]/div[2]/div[1]/article...body
# //*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/span/a
# //*[@id="archive-wrapper"]/div[4]/div[1]/div/div[3]

check the first page that the give path are properly work or not

In [5]:
# Wait for the page to load
time.sleep(3)  # Adjust based on page load time

# Extract article links
articles = driver.find_elements(By.XPATH, '//*[@id="archive-wrapper"]/div[4]/div[1]/div/div')
article_links = [article.find_element(By.XPATH, './div[1]/a').get_attribute('href') for article in articles]

# Loop through each article link and extract details
for link in article_links:
    driver.get(link)
    time.sleep(4)  # Ensure the page loads before extracting data

    try:
        # Extract article details
        title = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/h1').text
        source = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/span/a').text
        date = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/p').text
        body = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[2]/div[1]/article').text

        # Print extracted data
        print(f"Title: {title}")
        print(f"Source: {source}")
        print(f"Date: {date}")
        print(f"Article: {body[:500]}...")  # Print first 500 characters
        print("-" * 80)

    except Exception as e:
        print(f"Error extracting article data from {link}: {e}")



Title: India's Cornerstone Ventures hits first close of second fund at around $40m
Source: Vibhuti Sharma
Date: 15 January, 2025
Article: Indian SaaS-focused venture capital firm Cornerstone Ventures on Wednesday said it has hit the first close of its $200-million second investment vehicle at around $40 million.
The vehicle, launched in April last year, raised capital from domestic investors, including HNIs, family offices, corporates, and other institutions.
Founded by former Reliance executives Rajiv Vaishnav and Abhishek Prasad, Cornerstone expects to make the final close of the fund by December 2025.
Cornerstone raised $50 mil...
--------------------------------------------------------------------------------
Title: DEG commits to invest $40m in Jungle Ventures's fifth vehicle
Source: Quynh Nguyen
Date: 15 January, 2025
Article: DEG, the investment arm of German state-owned development bank KfW, has committed to investing $40 million in Jungle Ventures‘s fifth fund, which has a tar

check up the scripts and view the fomat of the first page data to preceding to the next page

In [9]:
import time
import csv
# Wait for the page to load
time.sleep(3)  # Adjust based on page load time

# Extract article links
articles = driver.find_elements(By.XPATH, '//*[@id="archive-wrapper"]/div[4]/div[1]/div/div')
article_links = [article.find_element(By.XPATH, './div[1]/a').get_attribute('href') for article in articles]

# Create and open a CSV file for writing
csv_filename = "scraped_articles.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Source", "Date", "Article Content", "URL"])  # Added URL column

    # Loop through each article link and extract details
    for link in article_links:
        driver.get(link)
        time.sleep(4)  # Ensure the page loads before extracting data

        try:
            # Extract article details
            title = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/h1').text.strip()
            source = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/span/a').text.strip()
            date = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/p').text.strip()
            body = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[2]/div[1]/article').text.strip().replace("\n", " ")  # Remove excessive newlines

            # Write the extracted data to the CSV file
            writer.writerow([title, source, date, body, link])

            print(f"Saved article: {title}")

        except Exception as e:
            print(f"Error extracting article data from {link}: {e}")

print(f"\nScraping complete! Data saved to {csv_filename}")


Saved article: India's Cornerstone Ventures hits first close of second fund at around $40m
Saved article: DEG commits to invest $40m in Jungle Ventures's fifth vehicle
Saved article: Indonesian fintech startup Skor Technologies secures $6.2m funding led by Argor Capital
Saved article: Indonesian AI startup Bythen secures $5m funding led by Vector Inc, Skystar Capital
Saved article: India's Boba tea and Korean fusion food brand Boba Bhai bags funding

Scraping complete! Data saved to scraped_articles.csv


to check up that the more button is work properly

In [12]:
click_count = 0
max_clicks = 3  # Maximum clicks

while click_count < max_clicks:
    try:
        # Locate the "More" button
        more_button = driver.find_element(By.XPATH, '//*[@id="archive-wrapper"]/div[5]/div/button')
        
        # Scroll into view and click
        driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", more_button)  # Click using JavaScript
        time.sleep(5)  # Wait for new articles to load
        
        click_count += 1
        print(f" 'More' button clicked {click_count} times")
        
        # Check if new articles are loaded
        articles = driver.find_elements(By.XPATH, '//*[@id="archive-wrapper"]/div[4]/div[1]/div/div')
        print(f"Total articles loaded: {len(articles)}")
        
        if len(articles) == 0:
            print(" No new articles found after clicking. Stopping.")
            break
        
    except (NoSuchElementException, ElementClickInterceptedException):
        print("⚠️ No 'More' button found or can't be clicked. Stopping.")
        break  # Stop if the button is missing or can't be clicked

print("\n✅ 'More' button test complete!")

✅ 'More' button clicked 1 times
🔄 Total articles loaded: 15
✅ 'More' button clicked 2 times
🔄 Total articles loaded: 25
✅ 'More' button clicked 3 times
🔄 Total articles loaded: 35

✅ 'More' button test complete!


fully code to scrape the last 2month data 

In [20]:
# Initialize WebDriver
driver = webdriver.Chrome()  # Change this if using a different browser
driver.get("https://www.dealstreetasia.com/section/venture-capital")
time.sleep(3)  # Wait for page to load

# Define the exact date limit (2 months ago)
two_months_ago = datetime.now() - timedelta(days=60)

# Create CSV file for writing
csv_filename = "full_scraped_articles.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Source", "Date", "Article Content", "URL"])  # CSV Header

    # Function to parse and reformat date
    def parse_and_format_date(date_str):
        try:
            parsed_date = datetime.strptime(date_str, "%d %B, %Y")  # Convert '14 January, 2025'
            return parsed_date.strftime("%d ,%m, %Y")  # Format to '14 ,01, 2025'
        except ValueError:
            return None  # Handle unexpected formats

    last_article_old = False  # Stop when an article older than 2 months is found
    last_scraped_index = 0  # Keep track of last scraped article index

    while not last_article_old:
        # Step 1: Extract all current article links
        articles = driver.find_elements(By.XPATH, '//*[@id="archive-wrapper"]/div[4]/div[1]/div/div')
        article_links = [article.find_element(By.XPATH, './div[1]/a').get_attribute('href') for article in articles]

        # Start from the last scraped index (continue from where we left off)
        for i in range(last_scraped_index, len(article_links)):
            link = article_links[i]
            driver.get(link)
            time.sleep(2)  # Wait for article page to load

            try:
                # Extract article details
                title = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/h1').text.strip()
                source = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/span/a').text.strip()
                date_text = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[1]/div/div[1]/p').text.strip()
                body = driver.find_element(By.XPATH, '//*[@id="disable-copy"]/div[2]/div[2]/div[1]/article').text.strip().replace("\n", " ")

                # Convert date and compare with the 2-month limit
                formatted_date = parse_and_format_date(date_text)
                article_date = datetime.strptime(formatted_date, "%d ,%m, %Y") if formatted_date else None

                if article_date and article_date < two_months_ago:
                    print(f"⏹️ Stopping: Found an article older than 2 months ({formatted_date})")
                    last_article_old = True
                    break  # Stop processing further

                # Save to CSV
                writer.writerow([title, source, formatted_date, body, link])
                print(f"✅ Saved article: {title} - {formatted_date}")

            except Exception as e:
                print(f" Error extracting article data from {link}: {e}")

            # Update last scraped index
            last_scraped_index = i + 1

            # Go back to the main page to scrape the next article
            driver.back()
            time.sleep(2)  # Wait for the main page to reload

        if last_article_old:
            break  # Stop clicking "More" if we found an old article

        # Step 2: Click the "More" button to load more articles
        try:
            more_button = driver.find_element(By.XPATH, '//*[@id="archive-wrapper"]/div[5]/div/button')
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(5)  # Wait for new articles to load

            # Update last_scraped_index to continue from where we left off
            last_scraped_index = len(article_links)

        except (NoSuchElementException, ElementClickInterceptedException):
            print(" No 'More' button found or can't be clicked. Stopping.")
            break  # Stop if the button is missing or can't be clicked

print(f"\n Scraping complete! Data saved to {csv_filename}")
driver.quit()


✅ Saved article: India's Cornerstone Ventures hits first close of second fund at around $40m - 15 ,01, 2025
✅ Saved article: DEG commits to invest $40m in Jungle Ventures's fifth vehicle - 15 ,01, 2025
✅ Saved article: Indonesian fintech startup Skor Technologies secures $6.2m funding led by Argor Capital - 14 ,01, 2025
✅ Saved article: Indonesian AI startup Bythen secures $5m funding led by Vector Inc, Skystar Capital - 14 ,01, 2025
✅ Saved article: India's Boba tea and Korean fusion food brand Boba Bhai bags funding - 13 ,01, 2025
✅ Saved article: PE major EQT anchors Cornerstone Robotics's $70m funding - 13 ,01, 2025
✅ Saved article: HK's Betatron Venture Group backs construction firm Varadise - 12 ,01, 2025
✅ Saved article: India Digest: GrayQuest, BluSmart in funding news - 10 ,01, 2025
✅ Saved article: Indian NBFC Infinity Fincorp raises $35m led by Jungle Ventures - 10 ,01, 2025
✅ Saved article: Healthcare SaaS startup Innovaccer raises $275m from B Capital, others - 10 ,01, 202